# **Εργασία** **2** **-** **Τεχνικές** **Εξόρυξης** **Δεδομένων**
Ονοματεπώνυμα: Αναστασία Ορφανουδάκη & Αλέξανδρος Κούτρας

Α.Μ: 1115202100129 & 1115202100073



# **ΜΕΡΟΣ 1ο**

Φορτώνουμε τη στήλη comments για κάθε μήνα του 2019, τα ενώνουμε σε ένα dataframe και τα προεπεξεργαζόματσε, αφαιρώντας τα σημεία
στίξης, μετατρέποντας όλους τους χαρακτήρες σε μικρούς, αφαιρώντας τα σύμβολα,
όπως hashtags, emoticons,emojis, links καθώς και τα stopwords από το σύνολο των
δεδομενων.

In [45]:
import pandas as pd

In [46]:
import re

In [47]:
import numpy as np

In [48]:
from nltk.corpus import stopwords

In [49]:
from transformers import pipeline

In [50]:
import matplotlib.pyplot as plt

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
import pickle

In [53]:
from gensim.models import Word2Vec

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [55]:
from sklearn.model_selection import cross_val_score, cross_val_predict

In [56]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [57]:
from sklearn.neighbors import KNeighborsClassifier

In [58]:
from sklearn.ensemble import RandomForestClassifier

In [59]:
from sklearn.svm import SVC

In [60]:
import gensim.downloader as api

In [61]:
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
pip install langdetect

In [63]:
from langdetect import detect, LangDetectException

In [64]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [66]:
# Διαβάζουμε τα αρχεία reviews.csv για να πάρουμε την στήλη σχόλιαγια τους μήνες Φεβρπυάριο, Μάρτιο και Απρίλιο
commentsFeb19 = pd.read_csv('/content/gdrive/MyDrive/data/data/2019/febrouary/reviews.csv')
commentsMarch19 = pd.read_csv('/content/gdrive/MyDrive/data/data/2019/march/reviews.csv')
commentsApril19 = pd.read_csv('/content/gdrive/MyDrive/data/data/2019/april/reviews.csv')

In [67]:
# Προσθέτουμε στήλη με τον μήνα για κάθε σετ δεδομένων
commentsFeb19['month'] = '02'
commentsMarch19['month'] = '03'
commentsApril19['month'] = '04'

In [68]:
# Ενώνουμε τα αρχεία
comments_2019 = pd.concat([commentsFeb19, commentsMarch19, commentsApril19], ignore_index=False)

# Αντικαθιστούμε τις NaN τιμές με 0
comments_2019.fillna(0, inplace=True )

# Αποθηκεύουμε το αρχείο σε csv
comments_2019.to_csv('comments_2019.csv', index = False)

Η ίδια διαδικασία για το 2023

In [69]:
# Διαβάζουμε τα αρχεία reviews.csv για να πάρουμε την στήλη σχόλιαγια τους μήνες Μάρτιο, Ιούνιο, Σεπτέμβριο
commentsMarch23 = pd.read_csv('/content/gdrive/MyDrive/data/data/2023/march/reviews.csv')
commentsJune23 = pd.read_csv('/content/gdrive/MyDrive/data/data/2023/june/reviews.csv')
commentsSept23 = pd.read_csv('/content/gdrive/MyDrive/data/data/2023/september/reviews.csv')

In [70]:
# Προσθέτουμε στήλη με τον μήνα για κάθε σετ δεδομένων
commentsMarch23['month'] = '03'
commentsJune23['month'] = '06'
commentsSept23['month'] = '09'

In [71]:
# Ενώνουμε τα αρχεία
comments_2023 = pd.concat([commentsMarch23, commentsJune23, commentsSept23], ignore_index=False)

# Αντικαθιστούμε τις NaN τιμές με 0
comments_2023.fillna(0, inplace=True )

# Αποθηκεύουμε το αρχείο σε csv
comments_2023.to_csv('comments_2023.csv', index = False)

## **1o bullet**

Χρησιμοποιώντας το HuggingFace επισημείωστε (annotation process) ως
προς το συναίσθημα (θετικό/αρνητικό/ουδέτερο) όσα περισσότερα comments
μπορείτε για το 2019. Στο τελικό αποτέλεσμα θα χρειαστεί να έχετε και ένα αναγνωριστικό id. Δηλαδή θα προκύψει ένα csv αρχείο (ή ένα dataframe) που θα έχει 3 στήλες (id, review,
sentiment)

In [72]:
stop_words = set(stopwords.words('english'))

# Παίρνουμε ένα ποσοστό από τα αρχεία
portion19 = comments_2019.sample(frac=0.01, random_state=42)
classifier = pipeline("sentiment-analysis")

def clean_text(text, chunk_size=500):
    # Διατηρούμε μόνο γράμματα και κενά και τα μετατρέπουμε σε μικρούς χαρακτήρες
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    cleaned_text = cleaned_text.lower()
    # Χρησιμοποιούμε την regex για να αντικαταστήσετε τους ειδικούς χαρακτήρες με κενό string
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    # Αφαιρούμε τα stopwords
    words = cleaned_text.split()
    filtered_words = [word for word in words if word not in stop_words]
    # Ενώνουμε τις φιλτραρισμένες λέξεις πίσω σε ένα string
    cleaned_text = ' '.join(filtered_words)
    chunked_text = [cleaned_text[i:i+chunk_size] for i in range(0, len(cleaned_text), chunk_size)]
    return chunked_text

def classify_sentiment(chunks):
    total_sentiment = 0

    for chunk in chunks:
        result = classifier(chunk)
        if result[0]['label'] == 'POSITIVE':
            total_sentiment += 1
        elif result[0]['label'] == 'NEGATIVE':
            total_sentiment -= 1

    # Καθορίζει το ολικό συναίσθημα με βάση το total_sentiment
    if total_sentiment > 0:
        return'POSITIVE'
    elif total_sentiment < 0:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

    return total_sentiment


# Αφαιρούμε τα μη Αγγλικά σχόλια
def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

portion19 = portion19[portion19['comments'].apply(is_english)]

portion19['review'] = portion19['comments'].apply(clean_text)
portion19['sentiment'] = portion19['review'].apply(classify_sentiment)

portion19['review'] = portion19['review'].apply(lambda x: ' '.join(x))

portion19 = portion19.loc[:, ['listing_id', 'review', 'sentiment', 'month']]

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


**2ο bullet**

Κάντε το ίδιο για τις κριτικές του 2023.


In [ ]:
stop_words = set(stopwords.words('english'))
portion23 = comments_2023.sample(frac=0.01, random_state=42)
classifier = pipeline("sentiment-analysis")

def clean_text(text, chunk_size=500):
    # Διατηρούμε μόνο γράμματα και κενά και τα μετατρέπουμε σε μικρούς χαρακτήρες
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    cleaned_text = cleaned_text.lower()
    # Χρησιμοποιούμε την regex για να αντικαταστήσετε τους ειδικούς χαρακτήρες με κενό string
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    # Αφαιρούμε τα stopwords
    words = cleaned_text.split()
    filtered_words = [word for word in words if word not in stop_words]
    # Ενώνουμε τις φιλτραρισμένες λέξεις πίσω σε ένα string
    cleaned_text = ' '.join(filtered_words)
    chunked_text = [cleaned_text[i:i+chunk_size] for i in range(0, len(cleaned_text), chunk_size)]
    return chunked_text

def classify_sentiment(chunks):
    total_sentiment = 0

    for chunk in chunks:
        result = classifier(chunk)
        if result[0]['label'] == 'POSITIVE':
            total_sentiment += 1
        elif result[0]['label'] == 'NEGATIVE':
            total_sentiment -= 1

    # Καθορίζει το ολικό συναίσθημα με βάση το total_sentiment
    if total_sentiment > 0:
        return'POSITIVE'
    elif total_sentiment < 0:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

    return total_sentiment

# Αφαιρούμε τα μη Αγγλικά σχόλια
def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

portion23 = portion23[portion23['comments'].apply(is_english)]

portion23['review'] = portion23['comments'].apply(clean_text)
portion23['sentiment'] = portion23['review'].apply(classify_sentiment)

portion23['review'] = portion23['review'].apply(lambda x: ' '.join(x))

portion23 = portion23.loc[:, ['listing_id', 'review', 'sentiment', 'month']]

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


**3ο bullet**

Συγκρίνετε το συνολικό συναίσθημα με την πάροδο του χρόνου (πχ ένα
ιστόγραμμα για κάθε χρόνο με την κατανομή των positive/negative/neutral).

In [ ]:
pivot_sentiment19 = portion19.pivot_table(index='month', columns='sentiment', values='review', aggfunc='count', fill_value=0)

# Φτιάχνουμε το γράφημα
pivot_sentiment19.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.xlabel('Year')
plt.ylabel('Number of Comments')
plt.title('Number of Positive, Negative, and Neutral Comments Per Year')
plt.legend(title='Sentiment')

plt.show()

In [ ]:
pivot_sentiment23 = portion23.pivot_table(index='month', columns='sentiment', values='review', aggfunc='count', fill_value=0)

# Φτιάχνουμε το γράφημα
pivot_sentiment23.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.xlabel('Year')
plt.ylabel('Number of Comments')
plt.title('Number of Positive, Negative, and Neutral Comments Per Year')
plt.legend(title='Sentiment')

plt.show()

**BONUS**

Συγκρίνετε το συναίσθημα ανά γειτονιά με την πάροδο του χρόνου

In [ ]:
portion19['year'] = 2019
portion23['year'] = 2023
combined_df = pd.concat([portion19, portion23], ignore_index=True)

In [ ]:
# Διαβάζομυε τα listing.csv αρχεία για όλους τους μήνες
listingFeb19 = pd.read_csv('/content/gdrive/MyDrive/data/data/2019/febrouary/listings.csv')
listingMarch19 = pd.read_csv('/content/gdrive/MyDrive/data/data/2019/march/listings.csv')
listingApril19 = pd.read_csv('/content/gdrive/MyDrive/data/data/2019/april/listings.csv')

# Kαι προσθέτουμε σε μία καινούρια στήλη τον μήνα
listingFeb19['month']='02'
listingMarch19['month']='03'
listingApril19['month'] = '04'

# Ενώνουμε και τα 3 αρχεία σε ένα καινούριο
train_2019 = pd.concat([listingFeb19, listingMarch19, listingApril19], ignore_index=False)
train_2019.dropna()

# Δημιουργούμε ενα αντίγραφο του train_2019 αλλά μόνο με μοναδικές τιμές με βάση το id
train_2019_unique = train_2019.drop_duplicates(subset=['id'])

In [ ]:
# Διαβάζομυε τα listing.csv αρχεία για όλους τους μήνες
listingMarch23 = pd.read_csv('/content/gdrive/MyDrive/data/data/2023/march/listings.csv')
listingJune23 = pd.read_csv('/content/gdrive/MyDrive/data/data/2023/june/listings.csv')
listingSept23 = pd.read_csv('/content/gdrive/MyDrive/data/data/2023/september/listings.csv')

# Kαι προσθέτουμε σε μία καινούρια στήλη τον μήνα
listingMarch23['month'] = '03'
listingJune23['month'] = '06'
listingSept23['month'] = '09'

# Ενώνουμε και τα 3 αρχεία σε ένα καινούριο
train_2023 = pd.concat([listingMarch23, listingJune23, listingSept23], ignore_index=False)
train_2023.dropna()
train_2023.rename(columns={'useless': 'neighbourhood'}, inplace=True)
train_2023.rename(columns={'neighbourhood': 'cleansed_neighbourhood'}, inplace=True)

# Δημιουργούμε ενα αντίγραφο του train_2023 αλλά μόνο με μοναδικές τιμές με βάση το id
train_2023_unique = train_2023.drop_duplicates(subset=['id'])

In [ ]:
# Ενώνουμε τα 2 train αρχεία
combined_train = pd.concat([train_2019_unique, train_2023_unique], ignore_index=True)

# Μεταονομάζουμε την στήλη id σε listing_id
combined_train.rename(columns={'id': 'listing_id'}, inplace=True)

In [ ]:
# Συγχωνεύουμε τα αρχεία με το listing_id
merged_dfs = pd.merge(combined_df, combined_train, on='listing_id', how='outer')

In [ ]:
merged_dfs_per_neighbourhood_and_year = merged_dfs.groupby(['neighbourhood', 'year', 'sentiment']).size().unstack(fill_value=0)

neighbourhoods = merged_dfs_per_neighbourhood_and_year.index.get_level_values('neighbourhood').unique()

labels = ['POSITIVE', 'NEGATIVE', 'NEUTRAL']
fig, ax = plt.subplots(figsize=(12, 8))

color_mapping = {
    (2019, 'POSITIVE'): 'green',
    (2019, 'NEGATIVE'): 'blue',
    (2019, 'NEUTRAL'): 'black',
    (2023, 'NEGATIVE'): 'red',
    (2023, 'POSITIVE'): 'orange',
    (2023, 'NEUTRAL'): 'pink'
}

bar_width = 0.35
years = [2019, 2023]

for i, year in enumerate(years):
    bar_positions = [pos + (i * bar_width) for pos in range(len(neighbourhoods))]

    for j, label in enumerate(labels):
        bar_heights = [merged_dfs_per_neighbourhood_and_year.loc[(neighbourhood, year), label] if (neighbourhood, year) in merged_dfs_per_neighbourhood_and_year.index else 0 for neighbourhood in neighbourhoods]
        ax.bar(bar_positions, bar_heights, bar_width, label=f'{year} - {label}', color=color_mapping[(year, label)], edgecolor='white', alpha=0.7)

# Προσαρμογή του άξονα x
ax.set_xticks([pos + bar_width for pos in range(len(neighbourhoods))])
ax.set_xticklabels(neighbourhoods, rotation=45, ha='right')

# Προσθήκη ετικετών και τίτλου
ax.set_xlabel('Neighbourhood')
ax.set_ylabel('Number of Reviews')
ax.set_title('Number of Reviews per Neighbourhood by Sentiment and Year')

# Προσθήκη υπομνήματος
ax.legend()

plt.tight_layout()
plt.show()

# **ΕΡΏΤΗΜΑ 2ο**
Στο ερώτημα αυτό θα χρειαστεί να δημιουργήσετε δύο νέα dataset:
*  Ένα αρχείο train.tsv (θα είναι το 80% των συνολικών data points) που θα περιέχει τα δεδομένα που θα χρησιμοποιήσετε για εκπαίδευση των μοντέλων
σας. Τα δεδομένα εκπαίδευσης περιέχουν την ένδειξη positive, negative ή
neutral

*  Ενα αρχείο test.tsv (το 20% των data points) το οποίο θα περιέχει τα δεδομένα
που θα χρησιμοποιήσετε για να δοκιμάσετε το μοντέλο σας και να κάνετε μία
πρόβλεψη. Πρέπει το μοντέλο σας να αποφασίσει για κάθε ένα από τα reviews
που υπάρχουν στο σύνολο των test αν εφράζει θετικό, αρνητικό ή ουδέτερο
συναίσθημα.

In [ ]:
train_data19, test_data19 = train_test_split(portion19, test_size=0.2, random_state=42)

# Αποθήκευση των συνόλων σε αρχεία train.tsv και test.tsv
train_data19.to_csv('train19.tsv', sep='\t', index=False)
test_data19.to_csv('test19.tsv', sep='\t', index=False)

Η ίδια διαδικασία για το 2023

In [ ]:
train_data23, test_data23 = train_test_split(portion23, test_size=0.2, random_state=42)

# Αποθήκευση των συνόλων σε αρχεία train.tsv και test.tsv
train_data23.to_csv('train23.tsv', sep='\t', index=False)
test_data23.to_csv('test23.tsv', sep='\t', index=False)

Ακολουθήστε τις οδηγίες που παρουσιάσαμε στο φροντιστήριο και ετοιμάστε τα
χαρακτηριστικά για κάθε review χρησιμοποιώντας:
*   Tf-idf
*   Word embeddings

In [ ]:
v19 = TfidfVectorizer()
train_tfidf19 = v19.fit_transform(train_data19['review'])
test_tfidf19 = v19.transform(test_data19['review'])

with open('tfidf_train19.pkl', 'wb') as f:
    pickle.dump(train_tfidf19, f)
with open('tfidf_test19.pkl', 'wb') as f:
    pickle.dump(test_tfidf19, f)

# Εκτυπώνουμε τον πίνακα TF-IDF για το train_data
print("\nTF-IDF Table for train data:")
print(train_tfidf19)

# Εκτυπώνουμε τον πίνακα TF-IDF για το test_data
print("\nTF-IDF Table for test data:")
print(test_tfidf19)

In [ ]:
# Word Embeddings χαρακτηριστικά (Word2Vec)
# Εκπαίδευση του μοντέλου Word2Vec
w2v_model19 = Word2Vec(sentences=train_data19['review'].apply(lambda x: x.split()), vector_size=100, window=5, min_count=1, workers=4)
w2v_model19.save("word2vec19.model")

# Δημιουργία χαρακτηριστικών Word Embeddings
def get_w2v_features(reviews, model, vector_size=100):
    features = []
    for review in reviews:
        words = review.split()
        word_vecs = [model.wv[word] for word in words if word in model.wv]
        if len(word_vecs) == 0:
            features.append(np.zeros(vector_size))
        else:
            features.append(np.mean(word_vecs, axis=0))
    return np.array(features)

train_w2v19 = get_w2v_features(train_data19['review'], w2v_model19)
test_w2v19 = get_w2v_features(test_data19['review'], w2v_model19)

# Αποθήκευση Word Embeddings χαρακτηριστικών
with open('w2v_train19.pkl', 'wb') as f:
    pickle.dump(train_w2v19, f)
with open('w2v_test19.pkl', 'wb') as f:
    pickle.dump(test_w2v19, f)

# Εκτύπωση των embeddings για τα πρώτα 5 reviews στο training set
print("First 5 training set embeddings:")
for i in range(5):
    print(f"Review {i+1} embedding:", train_w2v19[i])

# Εκτύπωση των embeddings για τα πρώτα 5 reviews στο test set
print("\nFirst 5 test set embeddings:")
for i in range(5):
    print(f"Review {i+1} embedding:", test_w2v19[i])

Η ίδια διαδικασία για το 2023

In [ ]:
v23 = TfidfVectorizer()
train_tfidf23 = v23.fit_transform(train_data23['review'])
test_tfidf23 = v23.transform(test_data23['review'])

with open('tfidf_train23.pkl', 'wb') as f:
    pickle.dump(train_tfidf23, f)
with open('tfidf_test23.pkl', 'wb') as f:
    pickle.dump(test_tfidf23, f)

# Εκτυπώνουμε τον πίνακα TF-IDF για το train_data
print("\nTF-IDF Table for train data:")
print(train_tfidf23)

# Εκτυπώνουμε τον πίνακα TF-IDF για το test_data
print("\nTF-IDF Table for test data:")
print(test_tfidf23)

In [ ]:
# Word Embeddings χαρακτηριστικά (Word2Vec)
# Εκπαίδευση του μοντέλου Word2Vec
w2v_model23 = Word2Vec(sentences=train_data23['review'].apply(lambda x: x.split()), vector_size=100, window=5, min_count=1, workers=4)
w2v_model23.save("word2vec23.model")

# Δημιουργία χαρακτηριστικών Word Embeddings
def get_w2v_features(reviews, model, vector_size=100):
    features = []
    for review in reviews:
        words = review.split()
        word_vecs = [model.wv[word] for word in words if word in model.wv]
        if len(word_vecs) == 0:
            features.append(np.zeros(vector_size))
        else:
            features.append(np.mean(word_vecs, axis=0))
    return np.array(features)

train_w2v23 = get_w2v_features(train_data23['review'], w2v_model23)
test_w2v23 = get_w2v_features(test_data23['review'], w2v_model23)

# Αποθήκευση Word Embeddings χαρακτηριστικών
with open('w2v_train23.pkl', 'wb') as f:
    pickle.dump(train_w2v23, f)
with open('w2v_test23.pkl', 'wb') as f:
    pickle.dump(test_w2v23, f)

# Εκτύπωση των embeddings για τα πρώτα 5 reviews στο training set
print("First 5 training set embeddings:")
for i in range(5):
    print(f"Review {i+1} embedding:", train_w2v23[i])

# Εκτύπωση των embeddings για τα πρώτα 5 reviews στο test set
print("\nFirst 5 test set embeddings:")
for i in range(5):
    print(f"Review {i+1} embedding:", test_w2v23[i])

Δοκιμάζουμε τους παρακάτω ταξινομητές (η δοκιμή/testing θα γίνει στα test
δεδομένα που έχουμε κρατήσει- εννοείται ότι τα test δεν θα χρησιμοποιηθούν για το
training!)

*  SVM
*  Random Forests
*  KNN

Δοκιμάστε τους ταξινομητές σας με τα χαρακτηριστικά TFID, και word embeddings.
Επίσης θα πρέπει να αξιολογήσετε και να καταγράψετε την απόδοση κάθε μεθόδου
χρησιμοποιώντας 10-fold Cross Validation χρησιμοποιώντας τις παρακάτω μετρικές:

*  Precision / Recall / F-Measure
*  Accuracy



In [ ]:
# Φόρτωση χαρακτηριστικών και ετικετών
with open('tfidf_train19.pkl', 'rb') as f:
    train_tfidf19 = pickle.load(f)
with open('tfidf_test19.pkl', 'rb') as f:
    test_tfidf19 = pickle.load(f)
with open('w2v_train19.pkl', 'rb') as f:
    train_w2v19 = pickle.load(f)
with open('w2v_test19.pkl', 'rb') as f:
    test_w2v19 = pickle.load(f)

y_train19 = train_data19['sentiment']
y_test19 = test_data19['sentiment']

# Ορισμός μοντέλων
models = {
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier()
}

# Εκπαίδευση και αξιολόγηση των μοντέλων με TF-IDF
for name, model in models.items():
    model.fit(train_tfidf19, y_train19)
    y_pred = model.predict(test_tfidf19)
    accuracy = accuracy_score(y_test19, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test19, y_pred, average='weighted')
    print(f'{name} with TF-IDF:')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}\n')

# Εκπαίδευση και αξιολόγηση των μοντέλων με Word Embeddings
for name, model in models.items():
    model.fit(train_w2v19, y_train19)
    y_pred = model.predict(test_w2v19)
    accuracy = accuracy_score(y_test19, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test19, y_pred, average='weighted')
    print(f'{name} with Word Embeddings:')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}\n')

Η ίδια διαδικασία και για το 2023

In [ ]:
# Φόρτωση χαρακτηριστικών και ετικετών
with open('tfidf_train23.pkl', 'rb') as f:
    train_tfidf23 = pickle.load(f)
with open('tfidf_test23.pkl', 'rb') as f:
    test_tfidf23 = pickle.load(f)
with open('w2v_train23.pkl', 'rb') as f:
    train_w2v23 = pickle.load(f)
with open('w2v_test23.pkl', 'rb') as f:
    test_w2v23 = pickle.load(f)

y_train23 = train_data23['sentiment']
y_test23 = test_data23['sentiment']

# Ορισμός μοντέλων
models = {
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier()
}

# Εκπαίδευση και αξιολόγηση των μοντέλων με TF-IDF
for name, model in models.items():
    model.fit(train_tfidf23, y_train23)
    y_pred = model.predict(test_tfidf23)
    accuracy = accuracy_score(y_test23, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test23, y_pred, average='weighted')
    print(f'{name} with TF-IDF:')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}\n')

# Εκπαίδευση και αξιολόγηση των μοντέλων με Word Embeddings
for name, model in models.items():
    model.fit(train_w2v23, y_train23)
    y_pred = model.predict(test_w2v23)
    accuracy = accuracy_score(y_test23, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test23, y_pred, average='weighted')
    print(f'{name} with Word Embeddings:')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}\n')


# **ΕΡΩΤΗΜΑ 3**

# **Ερώτημα 3**

Για τη στήλη comments και με χρήση word
embeddings φτιάξτε μία συνάρτηση η οποία θα
υπολογίζει το similarity μεταξύ vectors δύο
λέξεων (πχ cosine ή όποια άλλη μέθοδο
θέλετε)
Μπορείτε να χρησιμοποιήσετε οποιαδήποτε
μέθοδο φτιάχνει word embeddings (word2vec,
fastText, glove κτλ) η να χρησιμοποιήσετε έτοιμα pre-trained embeddings.
Στη συνέχεια για δύο λέξεις (τυχαίες που θα δίνει ένας χρήστης) και για μία
παράμετρο Ν να υπολογίζεται

(α) η σημασιολογική γειτονιά των δύο λέξεων και

(β)
το similarity των λέξεων το οποίο θα προκύπτει από τη γειτονιά με βάση τα τρία
παρακάτω σχήματα , δηλαδή 3 διαφορετικά similarities.

In [ ]:
# Load pre-trained Word2Vec embeddings
word2vec_vectors = api.load("word2vec-google-news-300")  # 300-dimensional Word2Vec embeddings

def compute_similarity(word1, word2, model):
    if word1 in model and word2 in model:
        vec1 = model[word1].reshape(1, -1)
        vec2 = model[word2].reshape(1, -1)
        similarity = cosine_similarity(vec1, vec2)[0][0]
        return similarity
    else:
        return None

def neighborhood_similarity(word1, word2, model, N):

    if word1 in model:
      neighborhood1 = model.most_similar(word1, topn=N)
    if word2 in model:
      neighborhood2 = model.most_similar(word2, topn=N)

    if not neighborhood1 or not neighborhood2:
        return None

    words1 = [w for w, _ in neighborhood1]
    words2 = [w for w, _ in neighborhood2]

    # Method 1: Average cosine similarity between all pairs of words in the neighborhoods
    cosine_similarities = []
    for w1 in words1:
        for w2 in words2:
            sim = compute_similarity(w1, w2, model)
            if sim is not None:
                cosine_similarities.append(sim)
    avg_cosine_similarity = np.mean(cosine_similarities) if cosine_similarities else 0

    # Method 2: Maximum cosine similarity between any pair of words in the neighborhoods
    max_cosine_similarity = np.max(cosine_similarities) if cosine_similarities else 0

    # Method 3: Minimum cosine similarity between any pair of words in the neighborhoods
    min_cosine_similarity = np.min(cosine_similarities) if cosine_similarities else 0

    return {
        'avg_cosine_similarity': avg_cosine_similarity,
        'max_cosine_similarity': max_cosine_similarity,
        'min_cosine_similarity': min_cosine_similarity
    }

def analyze_similarity(word1, word2, model, N_values):
    avg_similarities = []
    max_similarities = []
    min_similarities = []

    for N in N_values:
        similarities = neighborhood_similarity(word1, word2, model, N)
        if similarities:
            avg_similarities.append(similarities['avg_cosine_similarity'])
            max_similarities.append(similarities['max_cosine_similarity'])
            min_similarities.append(similarities['min_cosine_similarity'])
        else:
            avg_similarities.append(None)
            max_similarities.append(None)
            min_similarities.append(None)

    return avg_similarities, max_similarities, min_similarities

def display_neighborhoods(word, model, N):

    if word in model:
      neighborhood = model.most_similar(word, topn=N)

    if neighborhood:
        print(f"Semantic Neighborhood for '{word}' (N={N}):")
        for neighbor, similarity in neighborhood:
            print(f"  {neighbor}: {similarity:.4f}")
    else:
        print(f"Word '{word}' not found in the model.")

# Define the words and N values
word1 = "Alex"
word2 = "Anastasia"
N_values = [1, 5, 10, 20, 50, 100]

avg_similarities, max_similarities, min_similarities = analyze_similarity(word1, word2, word2vec_vectors, N_values)

# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(N_values, avg_similarities, label='Average Cosine Similarity', marker='o')
plt.plot(N_values, max_similarities, label='Maximum Cosine Similarity', marker='o')
plt.plot(N_values, min_similarities, label='Minimum Cosine Similarity', marker='o')
plt.xlabel('N (Neighborhood Size)')
plt.ylabel('Similarity')
plt.title('Similarity vs Neighborhood Size')
plt.legend()
plt.grid(True)
plt.show()

# Display neighborhoods for N = 10
N = 10

print(f"Neighborhoods for N = {N}:")
display_neighborhoods(word1, word2vec_vectors, N)
print()
display_neighborhoods(word2, word2vec_vectors, N)

